# Characters & Strings in Julia
---
* Created on 28 Aug 2023
* Created by Yooshin Oh (stevenoh0908@snu.ac.kr)
---
* <span class="mark">Documentation: https://docs.julialang.org/en/v1/manual/strings/</span>

## Introduction to Character Encoding

* C에서는 0 ~ 127의 1-byte Integer로 문자들을 Encoding하는 **ASCII** 규약이 있지만, ASCII 코드는 Non-english 문자들과 기타 여러 특수문자들을 표현하지 못하는 문제점이 있다.
* 그래서 특수문자를 포함하여 전 세계의 모든 문자들을 표현하는 Encoding 체계인 [**Unicode**](https://en.wikipedia.org/wiki/Unicode) 표준이 있다.

* Julia는 Traditional ASCII 코드는 물론, Unicode 표준도 Simple & Efficient하게 다룰 수 있도록 설계되어 있다.
* Julia에서 C-프로그래밍을 하는 것처럼 ASCII 코드만을 고려하여 코드를 짜더라도, Non-ASCII 문자를 만나면 Silently Corrupting Result보다는 명확한 Error Message를 띄워주므로 편리하다.

### Unicode 표준

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/New_Unicode_logo.svg/375px-New_Unicode_logo.svg.png" width="150"/>

* Unicode Consortium에서 제정한 전 세계 모든 언어들을 컴퓨터 내에서 표현하기 위한 Encoding 표준.

* 일반적인 언어들에서 사용되는 문자들은 물론, 최근에는 Emoji들도 Encoding한다.

* Unicode Standard는 다음 3개의 Encoding을 주요 Encoding으로 제시한다.
    * [UTF-8](https://en.wikipedia.org/wiki/UTF-8)
    * [UTF-16](https://en.wikipedia.org/wiki/UTF-16)
    * [UTF-32](https://en.wikipedia.org/wiki/UTF-32)

이하에서는 주요 Format인 UTF-8 Encoding을 주로 정리해둔다.

#### UTF-8 Encoding

* **UTF-8 (Unicode Transformation Format - 8-bit)**은 Unicode 표준에서 정의된 Encoding Format으로, 흔히 ASCII Encoding을 Extend한 Format으로 널리 이용된다.

* UTF-8 Encoding은 문자를 표현하는데 있어 고정된 byte 수를 사용하는 것이 아닌, 문자 범위에 따라 그 표현에 서로 다른 길이의 byte를 사용하는 **Variable-Length Character Encoding**을 채택하고 있다.

* UTF-8 Encoding은 문자들을 표현하는데 있어, 1 ~ 4 byte를 사용한다. (Variable-Length Character Encoding)
* UTF-8 Encoding은 ASCII Encoding과의 호환성을 염두에 두고 만들어졌기 때문에, 초기 128개 문자(First 128 Characters of Unicode)가 정확히 ASCII의 128개 문자(0 ~ 127)와 1:1 대응된다. 따라서 Valid ASCII Text는 자동으로 Valid UTF-8 Encoded Unicode Text가 된다.

**UTF-8 Encoding 방법**

* UTF-8 표준에서는 문자의 고유 번호(Code-Point)의 범우에 따라, 각 문자를 1 ~ 4 byte로 다음과 같은 방식으로 Encoding한다.

> **Code Point <-> UTF-8 Conversion**
>
> |First Code Point|Last Code Point|Byte 1|Byte 2|Byte 3|Byte 4|
> |:----:|:----:|:---:|:---:|:---:|:---:|
> |U+0000|U+007F|0xxxxxxx|None|None|None|
> |U+0080|U+07FF|110xxxxx|10xxxxxx|None|None|
> |U+0800|U+FFFF|1110xxxx|10xxxxxx|10xxxxxx|None|
> |U+10000|U+10FFFF|11110xxx|10xxxxxx|10xxxxxx|10xxxxxx|

* Code-Point에서 처음 128개 문자는 ASCII 문자들로, 1-byte로 Encoding된다.
* 다음 1,920개의 문자들은 2-byte로 Encoding되며, Latin-script Alphabet, IPA(International Phonetic Alphabet) extensions, Greek, Cyrillic, Coptic, Armenian, Hebrew, Arabic, Syriac, Thaana, N'Ko Alphabets들을 다루고 있다.
* 다음 61,440개의 문자들은 3-byte로 Encoding되며, Chinese, Japanese, Korean Character들이 다루어지고 있다.
* 그 이외의 1,048,576 Code Point에 대응하는 Emoji들의 문자들이나 Mathematical Symbol등의 문자들은 4-byte로 Encoding된다.
<br/><br/>
* UTF-8은 임의의 Code-Point에 상응하는 문자를 다음과 같은 방식으로 Encoding한다.
    1. Code-Point의 범위를 확인하고, Encoding에 이용할 byte-수를 결정한다.
    2. 다음 규칙에 따라 Leading-bits를 결정한다.
        * 1-byte Encoded인 경우 `0`
        * 2-byte Encoded인 경우 `110`
        * 3-byte Encoded인 경우 `1110`
        * 4-byte Encoded인 경우 `11110`
    3. Byte 2, 3, 4에서는 처음 2개 bit는 Continuing이라는 의미에서 `10`으로 시작한다.
    3. Code-Point를 이진수로 표현한다. 앞에서 Leading-bits와 Continuing bits를 제외하고 남은 자리들의 개수에 맞도록 앞에 0을 붙인다.
    4. Code-Point를 앞쪽 byte부터 순차대로 자리에 맞게 잘라서 집어넣는다.

> **Encoding 예시**
>
> 1. Euro Sign 문자 `€`의 Unicode Code Point는 `U+20AC`이다.
> 2. `U+20AC`는 `U+0800` ~ `U+FFFF` 사이에 있으므로, 3-byte Encoded Character이다.
> 3. 따라서 Byte 1의 Leading-bits는 `1110`이 되고, byte 1에는 4자리가 남는다.
> 4. Byte 2, 3의 경우 Continuing bits `10`을 제외하고 각 6자리가 남으므로, $4 + 6 + 6 = 16$ bit 자리가 있다. 따라서 `20AC`를 Binary Format으로 표현하되, 총 16자리가 되도록 표현하면 `0010 0000 1011 1101`이므로, 앞에서부터 각각 4자리, 6자리, 6자리 씩을 잘라서 다음과 같이 Encoding한다: `1110 0010 / 10 000010 / 10 111101` -> `1110 0010 1000 0010 1011 1101`

## Julia의 문자열에 대한 주요 High-Level Feature

* Julia는 Python과 유사하게 문자열이나 문자열 상수를 다루는데 있어서 Built-in Type인 `String` Type을 사용한다. 이 `String` 자료형은 모든 Unicode 문자를 `UTF-8` Encoding에 의해 기본적으로 처리한다. (`transcode` 내장함수를 이용하면 다른 Unicode Encoding (`UTF-16` or `UTF-32`) 으로 된 문자열들의 Encoding을 변경할 수 있다.
* Julia에서의 String Type은 모두 `AbstractString`의 Child Type이다. 그리고 다른 Package에서 String을 다루는 경우, `AbstractString`을 Implement하는 경우가 대부분이므로 임의의 문자열을 처리하는 Function을 만들고자 하는 경우, 그 인자의 Type을 `AbstractString`으로 정의해주는 것이 훨씬 좋다.
* C 계열 언어들과 유사하게, Julia는 Single Character를 표현하는 추상 자료형인 `AbstractChar`를 먼저 가지며, 그 밑에 Built-in Character Type인 `Char` 자료형이 있다. 이 `Char` 자료형은 UTF-8 인코딩에 따라 Unicode Character 1개를 표현하는 32-bit Primitive Type이다.
* Java에서와 마찬가지로, Julia에서 String은 Immutable이다. 즉, `AbstractString` 객체들은 모두 Immutable하여, 그 값을 바꿀 수 없다. 따라서 다른 String Value를 가지도록 한다면, '새로 만들어서' 할당해야 한다.

## Characters in Julia

* Julia의 `Char`는 UTF-8 Encoding에 따라 문자를 표현하는 4-byte Primitive Type으로, Special Leteral Representation과 Appropriate Arithmetic Behavior를 가지는 자료형으로, Unicode Code Point의 값(Numeric Value)로 Convert될 수 있는 자료형이다.
    * 그러나 위에서 설명했듯, Julia Package는 `AbstractChar`의 다른 sub-type들을 정의하여 다른 Text-Encoding에 따른 Operation을 Optimization 시킬 수 있다.
* 4-byte Fixed임에 주의하자. UTF-8 Encoding은 Variable-Length Encoding을 채택했지만, Julia의 `Char`는 4-byte를 이용한다. 즉, 1, 2, 3-byte Encoded인 경우 Leading bytes들을 모두 Zero로 채운다.

* **Julia에서 문자 상수의 표현은 C 문법을 따른다. 즉, 문자고 문자열이고 모두 `''`, `""`, `'''`나 `"""`를 사용하여 표현했던 Python과는 달리 문자 상수는 단따옴표 `''`로, 문자열 상수는 쌍따옴표 `""`로 표현한다.**

In [1]:
c = 'x'

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [2]:
typeof(c)

Char

* 위에서 설명한 대로, Julia의 `Char` Type은 그 Unicode Code Point에 상응하는 정수값으로 바꿀 수 있다.

In [3]:
c = Int('x')

120

In [4]:
typeof(c)

Int64

* 자명히 위 결과는 이 시스템이 64-bit Arch여서 `Int`의 기본 자료형이 `Int64`이기 때문에 `Int` 변환 결과가 `Int64`인 것이다. 32-bit Arch에서는 당연히 `Int32`로 될 것이다.

* 모든 Integer Value가 Unicode Format에서 Valid한 것은 아니다. 그러나 Julia는 Performance를 위해 Int -> Char Conversion에서 Every Character가 Valid한지 확인하지는 않는다. 변환 이후 그 문자가 Valid한지 확인하기 위해서는, `isvalid(T, x)` 내장함수를 사용해야 한다.

In [5]:
Char(120)

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [6]:
Char(0x110000)

'\U110000': Unicode U+110000 (category In: Invalid, too high)

In [7]:
isvalid(Char, 0x110000)

false

* 문자 상수를 입력할 때, `\u` Escape Sequence 뒤에 Unicode Code Point를 16진수로 입력하여 특정 문자를 지시할 수 있다. 소문자 `\u` 뒤에는 최대 4개의 16진수 문자를, 대문자로 한 `\U` 뒤에는 최대 8개의 16진수 문자를 쓸 수 있다. (그러나 UTF-8 Encoding에서 Valid한 Unicode 문자는 최대 6개의 16진수 문자로 표시됨: \U10FFFF`에 주의하자)

In [8]:
'\u0' # U+0000

'\0': ASCII/Unicode U+0000 (category Cc: Other, control)

In [9]:
'\u78' # U+0078

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [10]:
'\u2200' # U+2200

'∀': Unicode U+2200 (category Sm: Symbol, math)

In [11]:
'\U10ffff' # U+10FFFF

'\U10ffff': Unicode U+10FFFF (category Cn: Other, not assigned)

* Unicode Escape Form 외에도, C에서 사용하던 Traditional Escape Sequence들을 그대로 사용할 수 있다.

In [12]:
Int('\0') # Null Char (U+0000)

0

In [13]:
Int('\t') # Tab Char

9

In [14]:
Int('\n') # New Line

10

In [15]:
Int('\e')

27

In [16]:
Int('\x7f') # hexadecimal format expression

127

In [17]:
Int('\177') # octal format expression

127

* C에서 ASCII 코드에 따라 그러했던 것처럼, Julia의 경우도 `Char` type의 모든 문자들은 UTF-8 Encoding을 따르는 문자들이므로 정수형 자료라, 그 값을 비교할 수 있으며 동시에 정수와 연산할 수 있다.

In [18]:
'A' < 'a'

true

In [19]:
'A' <= 'a' <= 'Z'

false

In [20]:
'A' <= 'X' <= 'Z'

true

In [21]:
'x' - 'a'

23

In [22]:
'A' + 1

'B': ASCII/Unicode U+0042 (category Lu: Letter, uppercase)

## Strings in Julia

### String Basics

* Julia에서 **문자열 상수(String literals)는 Double Quote(`""`)나 Triple Double Quote(`"""`)로 표현된다.**

In [23]:
str = "Hello, world.\n"

"Hello, world.\n"

In [24]:
"""Contains "quote" characters"""

"Contains \"quote\" characters"

* 위와 같이 Python에서처럼 내부의 Double quote를 유지시키기 위해 Triple Double Quote를 사용하여 문자열 상수를 표현할 수 ㅣㅇㅆ다.

* 긴 문자열의 경우 Backslash(`\`) 뒤에 줄을 바꿈으로서 계속 이어나갈 수 있다. (단, 이 기능은 IJulia에서는 지원하지 않는다.)

In [29]:
"This is a long \
line"

LoadError: [91msyntax: invalid escape sequence[39m

### String Indexing

* Python이나 C와 마찬가지로, String의 각 위치의 Character에 `[]`를 사용한 Indexing으로 접근할 수 있다. **단, Matlab과 같이 Julia에서 Index는 1로 시작함에 주의한다.**

* Python과는 달리, Julia에서는 처음 Index와 마지막 Index를 지시하기 위해 `begin` 키워드와 `end` 키워드를 사용할 수 있다.

In [31]:
str = "Hello, World.\n"
str[begin]

'H': ASCII/Unicode U+0048 (category Lu: Letter, uppercase)

In [32]:
str = "Hello, World.\n"
str[1] # <=> str[begin]. Note that the index is starting from 1 in Julia, not zero.

'H': ASCII/Unicode U+0048 (category Lu: Letter, uppercase)

In [33]:
str = "Hello, World.\n"
str[6]

',': ASCII/Unicode U+002C (category Po: Punctuation, other)

In [34]:
str = "Hello, World.\n"
str[end]

'\n': ASCII/Unicode U+000A (category Cc: Other, control)

* String이나 배열 등 Integer로 Indexing 가능한 Julia Object들은 내장함수 `firstindex(indexable)`이나 `lastindex(indexable)`로 First Element Index나 Last Element Index를 Get할 수 있다.
* C와는 달리 `String` 자료형은 Last Index가 NULL Char가 아니다.

In [35]:
str = "Hello, World.\n"
firstindex(str)

1

In [36]:
str = "Hello, World.\n"
lastindex(str)

14

* `length` 내장함수를 이용하여 문자열의 길이를 측정할 수 있다. 당연히 결과는 1부터 Indexing이 시작하는 Julia의 특성상 `lastindex`의 실행 결과와 대부분 같다. 물론 Code Unit들을 여러 개 차지하는 Unicode 문자들이 있을 수 있어, 이들까지 고려한 `length` 내장함수로 문자열의 길이를 측정해주는 것이 훨씬 낫다.

In [38]:
str = "Hello, World.\n"
length(str)

14

* Julia에서 Indexable한 Object에서 Indexing에 사용하는 `begin`, `end` 키워드는 마치 상수처럼 동작하기 때문에, Indexing Operator `[]` 내부에서 연산할 수 있다.

In [39]:
str = "Hello, World.\n"
str[end-1]

'.': ASCII/Unicode U+002E (category Po: Punctuation, other)

In [40]:
str = "Hello, World.\n"
str[end÷2]

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

* 범위를 벗어나는 Indexing을 시도하면 (begin보다 작거나, end보다 큰 index를 사용한 Indexing을 시도하면) `BoundsError`가 raise된다.

In [41]:
str = "Hello, World.\n"
str[begin-1]

BoundsError: [91mBoundsError: attempt to access String[39m
[91m  at index [0][39m

In [42]:
str = "Hello, World.\n"
str[end+1]

BoundsError: [91mBoundsError: attempt to access String[39m
[91m  at index [15][39m

### String Slicing

* Python의 문법과 유사하게, Julia에서도 String을 Slicing할 수 있다. 그러나 Index가 Julia에서는 1부터 시작함에 주의한다.
* 단, **Python의 Slicing과는 달리, 끝 부분도 Slicing에 들어간다.**

In [43]:
str = "Hello, World.\n"
str[4:9] # Substring from 4th char to 9-th char

"lo, Wo"

* Julia에서 `String` 자료형을 Indexing하면 `Char`로 문자 1개가 반환되지만, `String` 자료형을 Slicing하면 `String`이 반환됨에 주의.

In [44]:
str = "Hello, World.\n"
str[6] # Returns Char

',': ASCII/Unicode U+002C (category Po: Punctuation, other)

In [45]:
str = "Hello, World.\n"
str[6:6] # Returns String

","

* Range Indexing(Slicing)을 쓰는 경우, 해당 부분 문자열을 원래 String에서 Copy하여 새로운 String을 만들어 반환한다. 그러다 이 대신에 부분 문자열을 Copy하지 않고, Original String에서 일부만 '보는' 식으로도 접근할 수 있다.
* `SubString` 객체 생성자를 이용하면 이미 있는 `String`에서 부분 문자열을 복사해내지 않고서도 이미 있는 `String`에서 일부만 추출할 수 있다. (*Alternatively, It is possible to create a view into a string, using the type `SubString`.*)
* `SubString` 생성자는 다음의 양식을 가진다: `SubString(String, start_idx, end_idx)`

In [46]:
str = "long string"
substr = SubString(str, 1, 4)

"long"

In [47]:
typeof(substr)

SubString{String}

* `@views` 매크로를 사용하면 String에 대한 Slicing 결과물을 `SubString` 객체로 바꿀 수 있다.

In [50]:
@views typeof(str[1:4])

String

<span class="burk">**Note**</span> IJulia에서는 `@views` 매크로가 제대로 먹히지 않는다. Julia REPL에서 위 코드를 실행하면 `SubString{String}`으로 정확히 나온다.

### Unicode (UTF-8) in Julia String

* Julia는 앞서 설명한 바대로 Unicode 표준을 따르는 String을 지원한다.

#### 문자열 상수 내에서 Unicode 문자 사용법

* `\u`를 사용하여 문자열 상수 내에서 Unicode 문자를 4자리 hexadecimal code point를 이용해 표현할 수 있다.
* `\U`를 사용하여 문자열 상수 내에서 Unicode 문자를 8자리 (그러나 실질적으로는 6자리) hexadecimal code point를 이용해 표현할 수 있다.

In [51]:
s = "\u2200 x \u2203 y"

"∀ x ∃ y"

#### Julia에서 String Index와 Code Units

* Julia에서 `String` 자료형에서 각각의 Index는 UTF-8 Encoding에서 Unicode 표준을 따르는 문자열의 각 Code Units (1 byte)을 지시한다. 따라서 Variable-length Encoding을 채택하는 UTF-8 Encoding에서, `String`의 어떤 Index는 유효한 문자의 시작이 아닐 수도 있다.
    * 이를테면, 3-byte 문자의 경우 3개 Index에 걸쳐 저장되므로, 이 중 1개만 유효한 문자 시작 Index이고, 그 이외는 유효하지 않은 문자 시작 Index가 된다.

* Julia UTF-8 `String`을 Indexing할 때, 유효하지 않은 문자의 Code Unit을 Indexing하면 `StringIndexError`가 raise가 된다.

In [52]:
s = "\u2200 x \u2203 y"
s[1] # \u2200은 U+2200으로 3-byte 문자로 s[1], s[2], s[3]에 걸쳐 저장된다.

'∀': Unicode U+2200 (category Sm: Symbol, math)

In [53]:
s[2]

StringIndexError: [91mStringIndexError("∀ x ∃ y", 2)[39m

In [54]:
s[3]

StringIndexError: [91mStringIndexError("∀ x ∃ y", 3)[39m

In [55]:
s = "\u2200 x \u2203 y"
s[4] # 따라서 4번째 byte가 다음 유효한 문자로 Blank 문자이다. 이는 ASCII 문자로 1-byte 차지.

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

* UTF-8 Encoding으로 된 문자열에서 다음 유효한 문자의 Code Unit에 상응하는 Index는 내장함수 `nextind(String, start_index, count)`를 사용하여 구할 수 있다.

In [56]:
s = "\u2200 x \u2203 y"
# s[1]이 첫 문자 \u2200의 시작 Code Unit이므로, 그 이후의 문자인 ' '의 시작 위치를 알려면 nextind(s, 1)을 쓰면 된다.
nextind(s, 1)

4

In [58]:
s = "\u2200 x \u2203 y"
# s[4]가 두 번째 문자 ' ' 의 시작 Code Unit이므로, 그 이후의 문자인 'x'의 시작 위치를 얻으려면 nextind(s, 4)로.
nextind(s, 4)

5

In [63]:
s = "\u2200 x \u2203 y"
# 첫 번째 문자에서 3개 뒤의 유효 문자를 얻으려면 인덱스를 nextind(s, begin, 3)
s[nextind(s, begin, 3)]

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

* UTF-8 Encoding에도 불구하고, **Indexing에서 `end`를 사용하면 항상 문자열에서 가장 마지막의 유효한 문자의 시작 Index를 지시한다.** (*`end` is always the last valid index into a collection*)
* 마찬가지로 `lastindex(String)`은 UTF-8 Encoding이어도 마지막으로 유효한 문자의 시작 Index를 돌려준다.

In [59]:
s = "\u2200 x \u2203 y"
s[end]

'y': ASCII/Unicode U+0079 (category Ll: Letter, lowercase)

In [60]:
s[end-1] # Valid

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

In [61]:
s[end-2]

StringIndexError: [91mStringIndexError("∀ x ∃ y", 9)[39m

* `prevind(String, start_index, count)`는 문자열 `String`에서 `start_index` 이전의 유효한 `count` 번째 Index를 돌려준다.

In [62]:
s[prevind(s, end, 2)]  # 문자열 s에서 끝 문자에서 2개 이전의 문자 get

'∃': Unicode U+2203 (category Sm: Symbol, math)

* 이렇기 때문에 UTF-8으로 Encoding된 `String` 자료에 대하여, 유효하지 않은 Byte까지 포함하여 Slicing을 행하면 `StringIndexError`가 raise된다.

In [64]:
s = "\u2200 x \u2203 y"
s[1:1]

"∀"

In [65]:
s[1:2]

StringIndexError: [91mStringIndexError("∀ x ∃ y", 2)[39m

In [66]:
s[1:3]

StringIndexError: [91mStringIndexError("∀ x ∃ y", 3)[39m

In [67]:
s[1:4]

"∀ "

* UTF-8 Encoding은 Variable-Length Encoding이기 때문에, `String`에서 `length(String)`을 이용해 얻은 문자의 개수 결과는 항상 `lastindex(String)`과 같다는 보장이 없다. 모든 문자가 ASCII 문자인 경우에만 두 함수의 결과가 같다. 따라서, `length(String) <= lastindex(String)`의 관계식이 항상 성립한다.

In [69]:
# UTF-8 Encoding으로 된 문자는 다음과 같은 Indexing으로 접근하면 안 된다. Indexing으로 접근해도 괜찮은 경우는 모든 문자가 1-byte ASCII 문자들인 경우뿐이다.
s = "\u2200 x \u2203 y"
for i = firstindex(s):lastindex(s)
    try
        println(s[i])
    catch
        # ignore the index error
    end
end

∀
 
x
 
∃
 
y


* 그러나 일반적으로는 Python-like하게 다음과 같이 순회하는 것이 일반적이다.

In [70]:
for c in s
    println(c)
end

∀
 
x
 
∃
 
y


* 위에서 설명한 UTF-8 Encoding의 Variable Length Encoding 때문에, 문자열 `String`을 조작하고자 하는 경우, Index를 1씩 증가 / 감소시키는 것이 아니라 `prevind()`나 `nextind()`를 사용해야 한다. 하지만 이들 내장함수 외에도, **`eachindex()` 내장함수를 사용하여 문자열에서 Valid한 문자 Index를 1차원 Vector로 가져올 수 있다.**

In [71]:
s = "\u2200 x \u2203 y"
collect(eachindex(s))

7-element Array{Int64,1}:
  1
  4
  5
  6
  7
 10
 11

In [72]:
eachindex(s)

Base.EachStringIndex{String}("∀ x ∃ y")

#### UTF-8 String에서 Code Unit Accessing

* UTF-8 Encoded String에서, `codeunit(String, index)`를 사용하면 index에 상응하는 Code Unit 1 byte를 16진수로 얻을 수 있다.

In [77]:
s = "\u2200 x \u2203 y"
# 위 s를 UTF-8 Encoding에 따라 변환하면 다음과 같다.
# \u2200: 3-byte, U+2200 -> 0010 0010 0000 0000
# ' ': 1-byte, U+0020 -> 0000 0000 0010 0000
# 'x': 1-byte, U+0078 -> 0000 0000 0111 1000
# ' ': 1-byte, U+0020 -> 0000 0000 0010 0000
# \u2203: 3-byte, u+2203 -> 0010 0010 0000 0011
# ' ': 1-byte, U+0020 -> 0000 0000 0010 0000
# 'y': 1-byte, U+0079 -> 000 0000 0111 1001
# 따라서 UTF-8로 Encoding하면 각 문자는:
# \u2200: 1110 0010 1000 1000 1000 0000 -> 0xe2 0x88 0x80
# ' ': 0010 0000 -> 0x20
# 'x': 0111 1000 -> 0x78
# ' ': 0010 0000 -> 0x20
# \u2203: 1110 0010 1000 1000 1000 0011 -> 0xe2 0x88 0x83
# ' ': 0010 0000 -> 0x20
# 'y': 0111 1001 -> 0x79
# 따라서 총 s의 Byte Sequence는 다음과 같다:
# 0xe2 0x88 0x80 0x20 0x78 0x20 0xe2 0x88 0x83 0x20 0x79
codeunit(s, 1) # 따라서 1번째 Code Unit은 0xe2

0xe2

In [78]:
# 2번째 Code Unit은 0x88
codeunit(s, 2)

0x88

* `ncodeunits(String)`을 이용해 문자열 `String`의 Code Unit의 개수를 얻을 수도 있고, `codeunits(String)` 내장함수를 이용해 모든 문자열 `String`의 Code Unit을 `AbstractVector{UInt8}` 자료형으로 얻을 수도 있다.

In [79]:
ncodeunits(s)

11

In [80]:
codeunits(s)

11-element Base.CodeUnits{UInt8,String}:
 0xe2
 0x88
 0x80
 0x20
 0x78
 0x20
 0xe2
 0x88
 0x83
 0x20
 0x79

#### Invalid Code Units for `String` In Julia

* Julia의 `String`은 Invalid한 Code Unit이 들어갈 수도 있다.
* 이 경우 Julia는 `Char` 자료형이 그러하듯 Invalid하다고 표기해준다. 마찬가지로 `isvalid` 내장함수로 유효성을 검사할 수 있다.

In [81]:
s = "\xc0\xa0\xe2\x88\xe2|"

"\xc0\xa0\xe2\x88\xe2|"

In [82]:
foreach(display, s)

'\xc0\xa0': [overlong] ASCII/Unicode U+0020 (category Zs: Separator, space)

'\xe2\x88': Malformed UTF-8 (category Ma: Malformed, bad data)

'\xe2': Malformed UTF-8 (category Ma: Malformed, bad data)

'|': ASCII/Unicode U+007C (category Sm: Symbol, math)

In [83]:
isvalid.(collect(s)) # Element-wise Function Call (. operation syntax)

4-element BitArray{1}:
 0
 0
 0
 1

In [84]:
s2 = "\xf7\xbf\xbf\xbf"
foreach(display, s2)

'\U1fffff': Unicode U+1FFFFF (category In: Invalid, too high)

### String Concatenation

* Julia에서는 `String` Concatenation을 `string()` 내장함수를 이용한 생성을 통해 진행할 수도 있다.

In [85]:
greet = "Hello"
whom = "world"
string(greet, ", ", whom, ".\n")

"Hello, world.\n"

* `+` 연산자를 사용할 수는 없다. (Java-unlike)

In [86]:
greet + ", " + whom + ".\n"

MethodError: [91mMethodError: no method matching +(::String, ::String)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:529[39m

* Julia에서는 `*` 연산자가 String Concatenation에 사용된다.
    * String Concatenation에 많이 사용되는 `+` 연산자가 아닌 `*`를 사용하는 이유는, `+` 연산자는 보통 수학에서 Commutative한 연산자이나 `*` 연산자는 그렇지 않기 때문이다(행렬곱). 문자열 합치기 연산은 Non-Commutative하므로 `*`를 채택하여 비교환성을 강조한 것이다.

In [87]:
greet * ", " * whom * ".\n"

"Hello, world.\n"

* Julia에서 String은 `UTF-8` Encoding을 사용함에 주의한다. 따라서 Invalid Character or String들을 이어붙인 결과가 Valid한 String이 될 수도 있다.

In [88]:
a, b = "\xe2\x88", "\x80"
c = string(a, b) # 0xe2 0x88 0x80은 U+2200으로 Valid한 UTF-8 Encoding에 따르는 문자이다.

"∀"

In [89]:
collect.([a, b, c])

3-element Array{Array{Char,1},1}:
 ['\xe2\x88']
 ['\x80']
 ['∀']

In [90]:
length.([a, b, c])

3-element Array{Int64,1}:
 1
 1
 1

### String Interpolation

* Python에서는 f-string 문법이나 format을 사용하여 문자열 내에 변수의 값을 문자열의 형태로 변환하여 넣을 수 있었으나, **Julia에서는 Bash-like 문자열 Interpolation 문법을 가진다.**
* **Julia에서 문자열 내에 특정 expression의 값을 넣으려면, Bash처럼 `$` 이후에 expression을 쓰면 된다.** 단, Bash와 달리 Expression을 감쌀 때는 `{}`가 아니라 `()`를 쓴다.

In [91]:
greet = "Hello"
whom = "World"
"$greet, $whom.\n"

"Hello, World.\n"

* 위와 같은 String Interpolation을 하면 Julia가 자동으로 이상을 `string(greet, ", ", whom, ".\n")`으로 변환하여 실행한다.

In [93]:
"1 + 2 = $(1+2)"

"1 + 2 = 3"

* Non-`AbstractString` Type들의 자료형은 상응하는 문자열로 자동 변환되어 Interpolation된다.

In [94]:
v = [1, 2, 3]
"v: $v"

"v: [1, 2, 3]"

* String Interpolation에서 `AbstractChar` 또는 `AbstractString`은 있는 그대로 들어간다.

In [96]:
c = 'x'
"hi, $c"

"hi, x"

* Julia에서는 String Interpolation 때문에, 문자열에서 `$`를 넣으려면 `\`로 Escape해야 한다.

In [97]:
print("I have \$100 in my account.\n")

I have $100 in my account.


### Triple-Quoted String Literals

* Python의 Pre-formatted string인 Triple-Quoted String Literals와 유사하게, Julia에서도 3중 쌍따옴표로 둘러싼 문자열은 Pre-formated로 들어간다.

In [98]:
str = """
    Hello,
    world.
"""

"    Hello,\n    world.\n"

* 이 때, Triple-Quoted String Literals에서 처음 Opening Triple-Quote 뒤 개행 문자는 자동으로 Strip된다.

In [99]:
"""
    Hello"""

"Hello"

In [101]:
"""Hello"""

"Hello"

In [104]:
"""

Hello"""

"\nHello"

* 그러나 Closing Triple-Quote 앞의 개행 문자는 무시되지 않는다.

In [102]:
"""
    Hello
"""

"    Hello\n"

* `\`를 사용하여 줄을 넘기는 경우 그 다음 줄 시작의 Indentation은 무시된다. (IJulia에서는 제대로 동작하지 않음. Julia REPL에서는 정상 동작함)

In [105]:
"""
    A very long\ 
    word""" # long 뒤의 \ + \n 때문에 word 앞의 \t와 개행은 무시된다.

LoadError: [91msyntax: invalid escape sequence[39m

* 아주 당연하게 Triple-Quoted String은 Escaping 없이 `"`을 포함할 수 있다.

In [106]:
""""Hello, World!" said he, after a very long journey."""

"\"Hello, World!\" said he, after a very long journey."

### String Common Operations

* Julia는 String에 대하여 다음의 Common Operation들을 지원한다.

|Operator|Expression|Description|
|:--:|:-----:|:--------------:|
|`<`|`String1 < String2`|Returns True if `String1` is less than `String2`, in lexicographically order|
|`>`|`String1 > String2`|Returns True if `String1` is greater than `String2`, in lexicographically order|
|`<=` or `≤`|`String1 <= String2`<br/>or<br/>`String1 ≤ String2`|Returns True if `String1` is less than or equal to `String2`, in lexicographically order|
|`>=` or `≥`|`String1 >= String2`<br/>or<br/>`String1 >= String2`|Returns True if `String1` is greater than or equal to `String2`, in lexicographically order|
|`==`|`String1 == String2`|Returns True if `String1` is equal to `String2`|
|`!=` or `≠`|`String1 != String2`<br/>or<br/>`String1 ≠ String2`|Returns True if `String1` is not equal to `String2`|

In [107]:
"apple" < "xylophone"

true

In [108]:
"abcd" == "xylophone"

false

In [109]:
"Hello, World." != "Goodbye, World."

true

In [110]:
"1 + 2 = 3" == "1 + 2 = $(1 + 2)"

true

### Few Useful Internal Functions for String

* Julia의 String은 `findfirst(Char, String)`을 통해 String의 처음부터 Char가 등장하는 첫 번때 위치 Index를 얻을 수 있다.
* 마찬가지로, `findlast(Char, String)`을 통해 String의 끝에서부터 앞으로 가면서 Char가 등장하는 첫 번째 위치 Index를 얻을 수 있다.

In [111]:
findfirst('o', "xylophone") # xylophone에서 o는 앞에서부터 4번째 문자.

4

In [112]:
findlast('o', "xylophone") # xylophone에서 o는 앞에서부터 7번째 문자에 뒤에서부터 세면 처음으로 등장.

7

In [114]:
findfirst('z', "xylophone") # 찾지 못하면 반환값이 없다.

* Julia의 String은 `findnext(Char, String, offset)`을 통해 offset에 해당하는 위치 뒤에서부터 Char가 String 내부에서 등장하는 위치 Index를 얻을 수 있다.
* 마찬가지로, `findprev(Char, String, offset)`을 통해 offset에 해당하는 위치 앞에서부터 Char가 String 내부에서 등장하는 위치 Index를 얻을 수 있다.

In [115]:
findnext('o', "xylophone", 1) # 1번째 index인 x 이후에서 o가 처음으로 등장하는 index는 4.

4

In [116]:
findnext('o', "xylophone", 5) # 5번째 index인 p 이후에서 o가 처음으로 등장하는 index는 7

7

In [117]:
findprev('o', "xylophone", 5) # 5번째 index인 p 이전에서 o가 처음으로 등장하는 index는 4

4

In [118]:
findnext('o', "xylophone", 8) # findnext, findprev는 결과를 찾지 못하면 Return값이 없다.

* 내장함수 `occursin(Substring, String)`을 사용하면 Substring이 String 내부에 등장하는지 아닌지를 T/F로 알 수 있다. (Substring은 Char여도 된다)

In [119]:
occursin("world", "Hello, world!")

true

In [120]:
occursin("o", "Xylophone")

true

In [121]:
occursin("a", "Xylophone")

false

In [122]:
occursin('o', "Xylophone") # Char도 Acceptable.

true

* `repeat(String, count)` 내장함수를 이용하여 특정 문자열 `String`을 `count`번 연이어 반복한 문자열을 얻을 수 있다. (Python이라면 `String * count`로 가능했지만, Julia는 `*`를 String Concatenation Operator로 채택한 관계로 그럴 수 없다.

In [123]:
repeat(".:Z:.", 10)

".:Z:..:Z:..:Z:..:Z:..:Z:..:Z:..:Z:..:Z:..:Z:..:Z:."

* Python에서의 `join` method와 비슷하게, Julia에서도 내장함수 `join(Container, *delims)`를 이용해 Container의 모든 문자 또는 문자열을 delims로 연결한 String을 얻을 수 있다.

In [124]:
join(["apples", "bananas", "pineapples"], ", ", " and ")

"apples, bananas and pineapples"

* 다음과 같은 함수들이 String Manipulation과 관련하여 내장함수로 유용하다.

|Function|Description|Example|
|:----:|:----------------:|:--------:|
|`firstindex(String)`|`String`에서 앞에서부터 처음으로 유효한 Byte Index를 Return|`firstindex("Hello")`|
|`lastindex(String)`|`String`에서 뒤에서부터 처음으로 유효한 Byte Index (앞에서부터 마지막으로 유효한 Byte Index)를 Return|`lastindex("Hello")`|
|`length(String)`|`String`의 길이를 Return|`length("Hello")`|
|`length(String, i, j)`|`String`의 `i`-th index에서 `j`-th index 사이에서 Valid Char의 개수를 Return|`length("Hello", 1, 3)`|
|`ncodeunits(String)`|`String`의 Code Unit의 총 개수를 Return|`ncodeunits("Hello")`|
|`codeunit(String, i)`|`String`의 `i`번째 byte의 Code Unit을 Return (16진수 형태)|
|`thisind(String, i)`|`String`에서 `i`번째 Code Unit이 가리키는 문자의 시작 Index를 Return|`thisind("\u2200", 2)`|
|`nextind(String, i, n=1)`|`String`에서 `i`번째 index 이후에 등장하는 `n`번째 문자의 시작 Index를 Return|`nextind("\u2200 x \u2203 y", 3, 2)`|
|`prevind(String, i, n=1)`|`String`에서 `i`번째 index 이전에 등장하는 `n`번째 문자의 시작 Index를 Return|`prevind("\u2200 x \u2203 y", 3, 2)`|

### Non-Standard String Literals

* Julia는 다음과 같은 3종의 비표준 문자열 상수를 지원한다.
    1. Regular Expressions (regex)
    2. Byte-array Literals
    3. Version Number Literals
* 이들 비표준 문자열 상수는 Python에서 f-String 또는 Raw String 문법과 유사하게, Regular String인 `""` 의 쌍따옴표 앞에 prefix가 붙어 특수 처리된다.

#### Regular Expressions

* Julia는 Perl-compatible Regular Expression 문법에 따라 (PCRE2 library) 정규 표현식을 지원한다. Julia에서 Regular Expression은 앞에 Prefix로 `r`을 붙인다.

In [125]:
re = r"^\s*(?:#|$)"
# Perl 문법에 따르면, (?:)는 Non-capturing Group을 형성하는 생성자로, Capturing Group이 아님을 지시하기 위해 사용한다.
# Non-Capturing Group은 Regex 상의 문법에서는 Capturing Group과 의미는 같지만, Capturing Group을 별도로 만들어 Memory에 저장하지는 않겠다는 의미이다.
# |는 OR의 의미이고, $는 앞에 뭔가가 있다면 그걸로 끝나는 것을 뜻하지만 단항으로 쓰이면 줄이 끝나는 것을 뜻한다.
# 따라서 상기 Regex는 Whitespace 문자가 0번 이상 반복되고, 그 뒤에 # 문자가 오거나 EOL(End-of-Line)이 오는 경우와 Matching된다.
# * Non-Capturing Group 관련해서는 다음 링크 참조: https://blog.rhostem.com/posts/2018-11-11-regex-capture-group#:~:text=%EB%85%BC%2D%EC%BA%A1%EC%B3%90%EB%A7%81%20%EA%B7%B8%EB%A3%B9(non%2Dcapturing%20group)&text=%EC%BA%A1%EC%B3%90%EB%A7%81%20%EA%B7%B8%EB%A3%B9%EC%9D%B4%20%EA%B2%B0%EA%B3%BC,%EB%A5%BC%20%EB%B3%80%EA%B2%BD%ED%95%98%EB%8A%94%20%EC%98%88%EC%A0%9C%EB%8B%A4.
typeof(re)

Regex

* Regex가 String과 Matching하는지 확인하려면, `occursin` 내장함수를 쓰면 된다. (Python에서처럼 `re` 모듈을 import해서 matches Method를 call하지 않아도 된다)

In [126]:
occursin(re, "not a comment")

false

In [127]:
occursin(re, "# comment")

true

In [134]:
occursin(re, " ")

true

* 그러나 `occursin` 내장함수는 해당 String에 Regex와 Match하는 부분이 있는지 없는지만을 true / false로 알려줄 뿐이므로, Matching된 String을 알고 싶다면 `match(Regex, String)` 내장함수를 사용해야 한다.

In [129]:
match(r"^\s*(?:#|$)", "not a comment")
# matching하는 게 없으면 아무것도 return하지 않는다. (정확히는 Core.nothing을 return)

In [133]:
match(r"^\s*(?:#|$)", "# a comment")

RegexMatch("#")

In [137]:
line = "# comment"
m = match(r"^\s*(?:#|$)", line)
if m === nothing
    println("not a comment")
else
    println("blank or comment")
end

blank or comment


* 위에서 보는 바와 같이, `match(Regex, String)` 함수는 만약 String에서 Regex와 Matching되는 부분이 있다면 `RegexMatch` 객체를 돌려주고, 그렇지 않다면 `Core.nothing`을 돌려준다.

* Julia에서 내장함수 `match(Regex, String)`은 3번째 arg로 start_idx를 줄 수 있다. 이 경우, Regex Matching 검사를 명시된 `start_idx`부터 행하게 된다.

In [138]:
m = match(r"[0-9]", "aaaa1aaaa2aaaa3", 1) # 처음부터 숫자들 중 first match return

RegexMatch("1")

In [139]:
m = match(r"[0-9]", "aaaa1aaaa2aaaa3", 6) # 앞에서부터 6번째 문자부터 숫자 first match return

RegexMatch("2")

In [140]:
m = match(r"[0-9]", "aaaa1aaaa2aaaa3", 11) # 앞에서부터 11번째 문자부터 숫자 first match return

RegexMatch("3")

##### `RegexMatch` 객체에서 몇 가지 주요 attr

* `RegexMatch` 객체의 다음 속성(Attribute)들은 꽤 유용하다.
    1. `RegexMatch.match`: Matching 결과에서 Match된 Entire Substring
    2. `RegexMatch.captures`: Matching 결과에서 Match된 Captured Substrings, as an array of Strings
    3. `RegexMatch.offset`: Matching 결과에서 Match가 시작되는 offset
    4. `RegexMatch.offsets`: Matching 결과에서 Match가 시작되는 offset들의 vector

* 만약 capturing이 실패했다면, `RegexMatch.captures`는 nothing으로 지정되고, `RegexMatch.offsets`은 0으로 지정된다. (Julia에서 Index는 1부터 시작함에 주의)

In [141]:
m = match(r"(a|b)(c)?(d)", "acd")

RegexMatch("acd", 1="a", 2="c", 3="d")

In [142]:
m.match

"acd"

In [144]:
m.captures # Capturing Group이 3개고, 각각의 Captued Group에 걸린 문자열들을 Array로 줌.

3-element Array{Union{Nothing, SubString{String}},1}:
 "a"
 "c"
 "d"

In [145]:
m.offset

1

In [146]:
m.offsets

3-element Array{Int64,1}:
 1
 2
 3

In [147]:
m = match(r"(a|b)(c)?(d)", "ad")

RegexMatch("ad", 1="a", 2=nothing, 3="d")

In [148]:
m.match

"ad"

In [149]:
m.captures

3-element Array{Union{Nothing, SubString{String}},1}:
 "a"
 nothing
 "d"

In [150]:
m.offset

1

In [151]:
m.offsets

3-element Array{Int64,1}:
 1
 0
 2

*Julia에서는 Unpacking(Python-like)을 지원하기 때문에, Array를 순회할 필요 없이 RegexMatch의 offsets나 captures의 결과를 풀어서 따로 저장할 수 있다.*

In [154]:
first, second, third = m.captures; first

"a"

* Perl Regex 문법에 따라, Capturing Group에 `?<...>`로 이름을 지정하였을 때, Indexing에서 `:Group_Name`이나 `Group_No`로 `RegexMatch`에 Indexing할 수 있다.

In [155]:
m = match(r"(?<hour>\d+):(?<minute>\d+)", "12:45")

RegexMatch("12:45", hour="12", minute="45")

In [156]:
m[:minute] # Indexing into RegexMatch, with the name of capturing group

"45"

In [157]:
m[2] # Indexing into RegexMatch, with the number of capturing group

"45"

* `replace(String, Regex => Replace_String)` 내장함수를 이용하여 특정 `String`에서 정규 표현식에 상응하는 부분을 특정 문자열로 대체할 수 있다. 이 때 `Replace_String`은 Prefix `s`로 표시해주어야 한다.
* 이 때, `replace` 내장함수에서 `Regex`에서 Capture Group의 이름을 설정한 경우, `Replace_String`에서 이들을 `\g<group_name>`의 형태로 사용할 수 있다.
* 또한, `replace` 내장함수의 `Regex`에서 Capture Group을 정한 경우, `Replace_String`에서 그 Capture Group의 번호를 이용해 `\group_no`의 형태로 사용할 수 있다.

In [159]:
replace("first second", r"(\w+) (?<agroup>\w+)" => s"\g<agroup> \1") #\1이 first-capture group인 first를 지시.

"second first"

* `replace` 내장함수의 `Replace_String`에서 Capture Group의 번호에 0을 쓰면, 전체 Capture Group들을 지시한다.

In [163]:
replace("first second", r"(\w+) (?<agroup>\w+)" => s"\0")

"first second"

* Numbered Capture Group은 Disambiguation을 위해 `\group_no` 외에도 `\g<group_no>`의 형태로도 reference 가능하다.

In [164]:
replace("a", r"." => s"\g<0>1")

"a1"

* Perl Regex에서 Regular Expression의 Behaviour을 Flag `i`, `m`, `s`, `x` 등을 조합하여 수정할 수 있는데, 이 Flag들은 Perl에서와 같은 의미를 가진다. 이들 Flag들은 Regex String의 뒤에 Postfix로 순차대로 붙여야 한다.
    * `i`: Do case-insensitive pattern matching.
    * `m`: Treat string as multiple lines. That is, change `^` and `$` from matching the start or end of the string to matching the start or end of any line anywhere within the string.
    * `s`: Treat string as single line. That is, change "." to match any character whatsover, even a newline, which normally it would not match.
    * `x`: Tells the regular expression parser to ignore most whitespace that is neither backslashed nor within a character class.

In [165]:
r"a+.*b+.*?d$"ism

r"a+.*b+.*?d$"ims

In [166]:
match(r"a+.*b+.*?d$"ism, "Goodbye,\nOh,angry,\nBad world\n")

RegexMatch("angry,\nBad world")

* Regex String에서 Interpolation이나 Unescaping은 일어나지 않는다. (단, "는 제외, "는 Escaping이 유일하게 허용된다)

In [167]:
x = 10

10

In [168]:
r"$x" # No Interpolation

r"$x"

In [169]:
"$x" # Normal String Literal, therefore Interpolation Occured.

"10"

In [170]:
r"\x" # No Escaping except for double quotation mark(")

r"\x"

In [171]:
"\x" # Normal String Literal, therefore Escaping Occured.

LoadError: [91msyntax: invalid escape sequence[39m

* Triple-quoted Regex String도 지원된다. 즉, `r"""..."""`도 지원된다.

* `Regex()` Constructor를 이용하면 Regular Expression을 Programmatic하게 만들 수 있다. 특히 String Concatenation Operator `*`와 함께 사용하면 큰 도움이 된다.

In [172]:
using Dates
d = Date(1962,7,10)
regex_d = Regex("Day " * string(day(d)))
match(regex_d, "It happended on Day 10")

RegexMatch("Day 10")

In [173]:
name = "Jon"
regex_name = Regex("[\"( ]\\Q$name\\E[\") ]") # interpolate value of name (\\은 \임에 주의)
# \Q는 Regex String에서 Regex가 아닌 문자열 상수로 해석되는 부분의 시작을 지시
# \E는 Regex String에서 Regex가 아닌 문자열 상수로 해석되는 부분의 끝을 지시
# 따라서 위 Regex에서 $name의 부분, 즉 Jon은 J, o, n이 아닌 Jon으로 있는 그대로 해석된다.
match(regex_name, " Jon ")

RegexMatch(" Jon ")

* <span class="mark">**Note**</span> All characters between the `\Q` and the `\E` are interpreted as literal characters (after string interpolation) -> Especially useful when interpolating, possibly malicious, user input. (Regex 문법에 따라 입력하는 경우 등...)

In [174]:
match(regex_name, "[Jon]") === nothing

true

#### Byte Array Literals

* String Literal 앞에 Prefix로 `b`를 붙이면 Julia는 이 문자열을 `String`이 아니라, 이 `String`을 Decode한 Byte Array로 취급한다. 즉, `UInt8` array로 취급한다. 정확히는 `CodeUnits{UInt8, String}`으로 취급한다.
    * ASCII를 비롯한 UTF-8 문자들은 있는 그대로 byte들로 바꾼다.
    * `\x` 또는 Octal Escape Sequence는 상응하는 byte들로 바꾼다.

In [175]:
b"DATA\xff\u2200"
# D, A, T, A는 각각 상응하는 ASCII 코드에 따른 1 byte들로 바뀐다.
# \xff는 0xff로 그대로.
# \u2200은 상응하는 3-byte unicode로 decode.

8-element Base.CodeUnits{UInt8,String}:
 0x44
 0x41
 0x54
 0x41
 0xff
 0xe2
 0x88
 0x80

* 위 byte array를 만들기 위해 사용한 문자열은 중간의 `0xff`가 Invalid하므로 Invalid UTF-8 String이 됨에 주의하자. 이처럼 Invalid UTF-8 String이더라도, 이런 식으로 byte array를 만드는 데 쓸 수 있다.

In [176]:
isvalid("DATA\xff\u2200")

false

* `CodeUnits{UInt8, Stirng}` Type은 Read-only처럼 작동하기 때문에, Modification을 위해 Standard Vector가 필요한 경우 `Vector{UInt8}` Constructor를 이용해 Type Casting할 수 있다.

In [177]:
x = b"123"

3-element Base.CodeUnits{UInt8,String}:
 0x31
 0x32
 0x33

In [178]:
x[1]

0x31

In [179]:
x[1] = 0x32 # Acts like read-only vector

ErrorException: [91msetindex! not defined for Base.CodeUnits{UInt8,String}[39m

In [180]:
Vector{UInt8}(x)

3-element Array{UInt8,1}:
 0x31
 0x32
 0x33

#### Version Number Literals

* Julia는 Non-Standard String Literal 중에서도 Version No를 쉽게 Express하기 위한 Version Number Literal을 제공하는데, 단순히 문자열 앞에 Prefix로 `v`를 붙이면 된다.
* Version Number Literal은 Julia Interpreter에 의해, 자동으로 `VersionNumber` 객체로 변환된다. 이 `VersionNumber` 객체는 sematic versioning 규약을 따른다.
    * **Sematic Versioning**: `major_ver.minor_ver.patch_ver-pre_release_alphanum+build_alphanum`
        * Ex) `0.2.1-rc1+win64`
            * Major Version: 0
            * Minor Version: 2
            * Patch Version: 1
            * Pre-release: rc1
            * Build: wind64
* Version Number Literal을 작성할 때, Major Version을 빼고 모든 요소는 Optional이다. 따라서, `v"0.2"`는 정확히 `v"0.2.0"`과 같고, `v"2"`는 `v"2.0.0"`과 같은 식이다.

* Julia는 Julia의 version을 `VersionNumber` 객체로 들고 있으며, 내부 상수 `VERSION`에 저장하고 있다.

* `VersionNumber` 객체는 비교 연산을 사용하여 쉽게 비교할 수 있다.

In [181]:
if v"0.2" <= VERSION < v"0.3-" # trailing - indicates before v0.3
    # do something specific to 0.2 release series
end

* `VERSION~ 상수 외에도, `VersionNumber` object들은 `Pkg` 모듈에서 패키지의 버전이나 의존성을 표시하기 위해 종종 사용된다.

### Raw String Literals

* Python의 Raw String과 유사하게, Julia도 Interpolation이나 Escaping을 무시하고 있는 그대로의 문자열을 저장할 수 있다. 단, Python과는 달리 `r` prefix가 Regex에 사용되기 때문에, `raw` prefix를 붙여야 한다.

* 단, Quotation Mark는 Escape되어야 하며, Quote Character 바로 앞에 오는 Backslash 도 Escaping되어야 한다. (그래야 raw 문자열 끝을 알 수 있음)

In [185]:
println(raw"\\ \\\"")
# \\는 그대로, 끝의 \\\"의 경우, \"는 쌍따옴표 하나(")로 처리되고, 그 앞의 \\는 쌍따옴표 앞의 \이므로 Escaping됨.

\\ \"
